In [7]:
import pandas as pd

In [8]:
data_url='https://archive.ics.uci.edu/ml/machine-learning-databases/car/car.data'

In [9]:
#getting dataset
df = pd.read_csv(data_url, header=None)
df

,0,1,2,3,4,5,6
0,vhigh,vhigh,2,2,small,low,unacc
1,vhigh,vhigh,2,2,small,med,unacc
2,vhigh,vhigh,2,2,small,high,unacc
3,vhigh,vhigh,2,2,med,low,unacc
4,vhigh,vhigh,2,2,med,med,unacc
...,...,...,...,...,...,...,...
1723,low,low,5more,more,med,med,good
1724,low,low,5more,more,med,high,vgood
1725,low,low,5more,more,big,low,unacc
1726,low,low,5more,more,big,med,good


In [10]:
# Column names
columns=['buying',
'maint',
'doors',
'persons',
'lug_boot',
'safety',
'class',
]

In [11]:
#getting the data with column names
df=pd.read_csv(data_url,header=None, names=columns)
df

,buying,maint,doors,persons,lug_boot,safety,class
0,vhigh,vhigh,2,2,small,low,unacc
1,vhigh,vhigh,2,2,small,med,unacc
2,vhigh,vhigh,2,2,small,high,unacc
3,vhigh,vhigh,2,2,med,low,unacc
4,vhigh,vhigh,2,2,med,med,unacc
...,...,...,...,...,...,...,...
1723,low,low,5more,more,med,med,good
1724,low,low,5more,more,med,high,vgood
1725,low,low,5more,more,big,low,unacc
1726,low,low,5more,more,big,med,good


In [12]:
from sklearn.preprocessing import OrdinalEncoder
from numpy import asarray

oe = OrdinalEncoder()
for i in df.columns:
    cd=asarray(df[i])
    df[i]=oe.fit_transform(cd.reshape(-1,1))

df.sample(10)

,buying,maint,doors,persons,lug_boot,safety,class
305,3.0,2.0,3.0,0.0,0.0,0.0,2.0
754,0.0,2.0,3.0,2.0,0.0,2.0,0.0
1146,2.0,2.0,2.0,1.0,1.0,1.0,2.0
856,0.0,1.0,3.0,2.0,2.0,2.0,2.0
587,0.0,0.0,1.0,2.0,2.0,0.0,0.0
707,0.0,2.0,2.0,0.0,1.0,0.0,2.0
461,0.0,3.0,1.0,0.0,2.0,0.0,2.0
615,0.0,0.0,2.0,2.0,1.0,1.0,2.0
1565,1.0,2.0,1.0,2.0,0.0,0.0,3.0
363,3.0,1.0,1.0,1.0,1.0,1.0,2.0
